# กราฟวงกลมยอดขายผลไม้ (แนว OOP)

สมุดโน้ตนี้สาธิตการออกแบบโปรแกรมเชิงวัตถุ (OOP) สำหรับการแสดงผลข้อมูลยอดขายผลไม้เป็น **กราฟวงกลม (Pie Chart)** ด้วย `matplotlib`.

**สิ่งที่มีในไฟล์นี้**
- โมเดลข้อมูล `Fruit`
- ชั้นจัดการชุดข้อมูล/ตรรกะ `FruitDataset`
- ชั้นวาดกราฟ `FruitPlotter` พร้อมเมธอด `plot_pie`
- ตัวอย่างการใช้งานและแนวคิด OOP ที่ใช้


## แนวคิด OOP ที่ใช้
1. **Encapsulation (การห่อหุ้ม)**: เก็บสถานะและพฤติกรรมไว้ในคลาส `Fruit`, `FruitDataset`, `FruitPlotter` แยกความรับผิดชอบชัดเจน
2. **Single Responsibility**: แต่ละคลาสทำหน้าที่เดียว
   - `Fruit` = โครงสร้างข้อมูลผลไม้ 1 รายการ
   - `FruitDataset` = ตรรกะจัดการชุดข้อมูล (ดึงชื่อ, ยอดขาย, จัดกลุ่ม, เรียง)
   - `FruitPlotter` = วาดกราฟ/แสดงผล
3. **Reusability/Extensibility**: สามารถเพิ่มกราฟอื่น (bar/line/pie), เพิ่มเมธอดสรุปใน `FruitDataset` ได้โดยไม่กระทบส่วนอื่นมาก


In [ ]:
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Iterable, Dict
from collections import defaultdict

@dataclass(frozen=True)
class Fruit:
    name: str
    origin: str
    unit_sold: int

class FruitDataset:
    def __init__(self, fruits: Iterable[Fruit]):
        self._fruits: List[Fruit] = list(fruits)

    def names(self) -> List[str]:
        return [f.name for f in self._fruits]

    def sales(self) -> List[int]:
        return [f.unit_sold for f in self._fruits]

    def group_sales_by_origin(self) -> Dict[str, int]:
        agg = defaultdict(int)
        for f in self._fruits:
            agg[f.origin] += f.unit_sold
        return dict(agg)

    def sort_by_sales(self, descending: bool = True) -> "FruitDataset":
        return FruitDataset(sorted(self._fruits, key=lambda f: f.unit_sold, reverse=descending))

class FruitPlotter:
    @staticmethod
    def plot_pie(ds: FruitDataset, title: str = "Fruit Sales Share"):
        labels = ds.names()
        sales = ds.sales()
        plt.figure()
        plt.pie(sales, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title(title)
        plt.tight_layout()
        plt.show()

    @staticmethod
    def plot_origin_pie(ds: FruitDataset, title: str = "Sales Share by Origin"):
        grouped = ds.group_sales_by_origin()
        labels = list(grouped.keys())
        sales = list(grouped.values())
        plt.figure()
        plt.pie(sales, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.title(title)
        plt.tight_layout()
        plt.show()


## เตรียมข้อมูลตัวอย่าง

In [ ]:
fruits = [
    Fruit("Apple",  "USA",      40),
    Fruit("Banana", "Thailand", 70),
    Fruit("Mango",  "India",    55),
    Fruit("Orange", "Spain",    30),
    Fruit("Durian", "Thailand", 15),
]
ds = FruitDataset(fruits)
len(ds.names()), ds.sales()


## กราฟวงกลม: สัดส่วนยอดขายรายผลไม้

In [ ]:
FruitPlotter.plot_pie(ds, title="Fruit Sales Share (Per Fruit)")

## กราฟวงกลม: สัดส่วนยอดขายตามประเทศผู้ผลิต

In [ ]:
FruitPlotter.plot_origin_pie(ds, title="Fruit Sales Share (By Origin)")

### หมายเหตุการใช้งาน
- ใช้ `matplotlib` และไม่ระบุสีเอง ปล่อยให้ใช้ค่าเริ่มต้น
- ควรแสดงกราฟทีละรูปต่อเซลล์ (หนึ่งกราฟต่อหนึ่งคำสั่งแสดงผล) เพื่ออ่านง่าย
- หากต้องการเรียงก่อนทำกราฟ: `sorted_ds = ds.sort_by_sales(True)` แล้วส่งเข้า `plot_pie(sorted_ds)`
